In [75]:
import os

import pandas as pd
from pyproj import Proj,transform

from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import STAMEN_TONER
from bokeh.models import ColumnDataSource, HoverTool, LabelSet

In [85]:

def convertCoords(row):
    inProj  = Proj(init='epsg:4326')
    outProj = Proj(init='epsg:3857')
    x2,y2 = transform(inProj,outProj,row['stop_lon'],row['stop_lat'])
    return pd.Series({'stop_m-lon':x2,'stop_m-lat':y2})


def plot_stops(network_dir, stop_labels):

    stoptimes_df      = pd.read_csv(os.path.join(network_dir, 'stop_times.txt'), usecols=['trip_id','stop_id','stop_sequence'])
    stops_df          = pd.read_csv(os.path.join(network_dir, 'stops.txt'), usecols=['stop_id','stop_name','stop_lat','stop_lon'])
    xy_merc           = stops_df.apply(convertCoords, axis=1)
    stops_df          = pd.concat([stops_df, xy_merc], axis=1)
    trip_stops_df     = pd.merge(stops_df, stoptimes_df, on='stop_id', how='left')
    trip_count_df     = trip_stops_df.groupby(['stop_id']).size().reset_index(name='trips')
    stops_size_df     = pd.merge(stops_df, trip_count_df, on='stop_id')
    
    size_factor       = stops_size_df['trips'].max()/70
    if stops_size_df['trips'].max() < 10: size_factor = 0.05
       
    source = ColumnDataSource(data=dict(
                         x    =list(stops_size_df['stop_m-lon']), 
                         y    =list(stops_size_df['stop_m-lat']),
                         trips=list(stops_size_df['trips']),
                         size =list(stops_size_df['trips']/size_factor),
                         offs =list(stops_size_df['trips']/(size_factor*2)),
                         name =list(stops_size_df['stop_name'])))

    hover = HoverTool(tooltips=[("stop", "@name"),("trips","@trips") ])


    p = figure(x_axis_type="mercator", y_axis_type="mercator")
   # tools=[hover, pan, 'wheel_zoom','save']
    p.add_tile(STAMEN_TONER)
    
    
    p.circle(x = 'x',
         y = 'y',
         source = source,
         size = 'size',
         fill_color="#FF1493",
         line_color="#FF69B4",
         fill_alpha=0.15,
         line_alpha=0.40)
    
    labels = LabelSet(x='x', y='y', text='name', level='glyph',text_color="#FF1493",
              x_offset='offs', y_offset='offs', source=source, render_mode='canvas')
    
    if stop_labels: p.add_layout(labels)
    
    return p


In [252]:
import ipywidgets as widgets

HOME_DIR = r"/Users/elizabeth/Documents/urbanlabs/MTC/SHRP2/task4"
@widgets.interact(network_dir = os.listdir(HOME_DIR),stop_labels=False)
def plot_net(network_dir,stop_labels):
    p = plot_stops(os.path.join(HOME_DIR,network_dir), stop_labels)
    p.title.text = "Example Network: "+ network_dir
    output_notebook()
    show(p)

interactive(children=(Dropdown(description=u'network_dir', options=('CHTS Run 7', 'network_draft1.14_fare', '.DS_Store', 'CHTS_fasttrips_demand_v0.5', 'readme-calibration.md', 'calibration-template.md', 'CHTS Run 3', 'SFO_SOMA_BART', 'trace_2201852', 'network_draft1.14_fare_clockhour', 'CHTS_fasttrips_demand_v0.5_sample', 'Route Choice with Stochastic Vehicle Arrivals-ELO.docx'), value='CHTS Run 7'), Checkbox(value=False, description=u'stop_labels'), Output()), _dom_classes=('widget-interact',))

# Routes


In [227]:


def plot_routes(network_dir, sel_routes=[]):
    print network_dir, sel_routes
    stoptimes_df      = pd.read_csv(os.path.join(network_dir, 'stop_times.txt'), usecols=['trip_id','stop_sequence','arrival_time'])
    firststop_df      = stoptimes_df[stoptimes_df['stop_sequence']==1]
    trips_df          = pd.read_csv(os.path.join(network_dir, 'trips.txt'))
    firststop_df      = pd.merge(firststop_df, trips_df, on='trip_id', how='left')
    #firststop_df      = firststop_df[firststop_df['route_id'].isin(sel_routes)]
    
    if 'direction_id' not in firststop_df:
        firststop_df['direction_id'] = 0
    
    firststop_df.loc[firststop_df['direction_id']==0,'route_id-dir'] = firststop_df.loc[firststop_df['direction_id']==0,'route_id']+"-Out"
    firststop_df.loc[firststop_df['direction_id']==1,'route_id-dir'] = firststop_df.loc[firststop_df['direction_id']==1,'route_id']+"-In"

    routes = sorted(list(set(firststop_df["route_id-dir"])))

    out_df = firststop_df[firststop_df['direction_id']==0]
    in_df  = firststop_df[firststop_df['direction_id']==1]

    route_in = ColumnDataSource(data=dict(
                   x    =[ pd.to_datetime(t) for t in list(in_df['arrival_time'])], 
                   y    =list(in_df['route_id-dir']),
                   t    =list(in_df['arrival_time']),
                   name =list(in_df['trip_id'])))

    route_out = ColumnDataSource(data=dict(
                   x    =[ pd.to_datetime(t) for t in list(out_df['arrival_time'])], 
                   y    =list(out_df['route_id-dir']),
                   t    =list(out_df['arrival_time']),
                   name =list(out_df['trip_id'])))

    tips=[
        ("trip", "@name"),
        ("time", "@t"),
    ]

    p_route = figure(x_axis_type="datetime",width=900, y_range=routes,
                     title='Route Timing')

    p_route.circle(x = 'x',
             y = 'y',
             source = route_in,
             size = 20,
             fill_color="#0892D0",
             line_color="#0892D0",
             fill_alpha=.5,
             line_alpha=0.40)

    p_route.circle(x = 'x',
             y = 'y',
             source = route_out,
             size = 20,
             fill_color="#FF1493",
             line_color="#FF69B4",
             fill_alpha=.5,
             line_alpha=0.40)

    p_route.add_tools(HoverTool(tooltips=tips))

    return p_route



In [250]:
HOME_DIR    = os.path.join(os.getcwd(), '..', "fasttrips", "Examples","networks" )

default=os.listdir(HOME_DIR)[2]

net_widget = widgets.Dropdown(
    options=os.listdir(HOME_DIR),
    value=default,
    description='Network:',
    disabled=False,
)

rte_widget = widgets.SelectMultiple(
    options=sorted(list(set(pd.read_csv(os.path.join(HOME_DIR,default, 'trips.txt'))['route_id']))),
    value=[],
    description='Routes',
    disabled=False
)

def update_rte(*args):
    rte_widget.options = sorted(list(set(pd.read_csv(os.path.join(HOME_DIR,net_widget.value, 'trips.txt'))['route_id'])))

net_widget.observe(update_rte, 'value')

def plot_rt(net_dir, sel_routes = []):
    p_route = plot_routes(os.path.join(HOME_DIR,default), sel_routes=routes)
    output_notebook()
    show(p_route)
    
widgets.interact(plot_rt, net_dir = net_widget, sel_routes = rte_widget)

   


interactive(children=(Dropdown(description=u'Network:', index=2, options=('simple', 'bunny_hop', 'psrc_1_1', 'many_bunny_hops'), value='psrc_1_1'), SelectMultiple(description=u'Routes', options=('1_1001', '1_1002', '1_1003', '1_1004', '1_1005', '1_1007', '1_1008', '1_1009', '1_1010', '1_1011', '1_1012', '1_1013', '1_1014', '1_1015', '1_1016', '1_1017', '1_1018', '1_1019', '1_1021', '1_1022', '1_1023', '1_1024', '1_1025', '1_1026', '1_1027', '1_1028', '1_1030', '1_1031', '1_1033', '1_1034', '1_1035', '1_1036', '1_1037', '1_1038', '1_1039', '1_1041', '1_1042', '1_1043', '1_1044', '1_1045', '1_1046', '1_1048', '1_1049', '1_1051', '1_1053', '1_1054', '1_1055', '1_1056', '1_1057', '1_1060', '1_1064', '1_1065', '1_1066', '1_1067', '1_1068', '1_1070', '1_1071', '1_1072', '1_1073', '1_1074', '1_1075', '1_1076', '1_1077', '1_1079', '1_1098', '1_1099', '1_1101', '1_1102', '1_1105', '1_1106', '1_1107', '1_1110', '1_1111', '1_1113', '1_1114', '1_1116', '1_1118', '1_1119', '1_1120', '1_1121', '1_1122', '1_1123', '1_1124', '1_1125', '1_1128', '1_1129', '1_1131', '1_1132', '1_1133', '1_1134', '1_1139', '1_1140', '1_1143', '1_1148', '1_1149', '1_1150', '1_1152', '1_1153', '1_1154', '1_1155', '1_1156', '1_1157', '1_1158', '1_1159', '1_1161', '1_1162', '1_1164', '1_1166', '1_1167', '1_1168', '1_1169', '1_1173', '1_1174', '1_1175', '1_1177', '1_1179', '1_1180', '1_1181', '1_1182', '1_1183', '1_1187', '1_1190', '1_1192', '1_1196', '1_1197', '1_1200', '1_1201', '1_1202', '1_1203', '1_1204', '1_1205', '1_1206', '1_1207', '1_1208', '1_1209', '1_1210', '1_1211', '1_1212', '1_1214', '1_1215', '1_1216', '1_1217', '1_1218', '1_1219', '1_1221', '1_1222', '1_1225', '1_1229', '1_1230', '1_1232', '1_1233', '1_1234', '1_1236', '1_1238', '1_1240', '1_1242', '1_1243', '1_1244', '1_1245', '1_1247', '1_1248', '1_1249', '1_1250', '1_1251', '1_1252', '1_1253', '1_1255', '1_1256', '1_1257', '1_1260', '1_1261', '1_1265', '1_1266', '1_1268', '1_1269', '1_1271', '1_1272', '1_1273', '1_1277', '1_1291', '1_1301', '1_1303', '1_1304', '1_1306', '1_1308', '1_1311', '1_1312', '1_1316', '1_1330', '1_1331', '1_1342', '1_1345', '1_1346', '1_1347', '1_1348', '1_1355', '1_1358', '1_1372', '1_1373', '1_1600', '1_1661', '1_1885', '1_1886', '1_1888', '1_1889', '1_1890', '1_1891', '1_1892', '1_1901', '1_1903', '1_1908', '1_1909', '1_1913', '1_1915', '1_1917', '1_1918', '1_1919', '1_1921', '1_1926', '1_1927', '1_1929', '1_1935', '1_1941', '1_1982', '1_1986', '1_1987', '1_1988', '1_1989', '1_1994', '1_1995', '1_2001', '1_2002', '1_2003', '1_2010', '1_2011', '1_2013', '1_2016', '1_2026', '1_2028', '1_2041', '1_2042', '1_2045', '1_2048', '1_2051', '1_2052', '1_2053', '1_2054', '1_2055', '1_2056', '1_2057', '1_2059', '1_2060', '1_2061', '1_2100', '1_2102', '1_2113', '1_2202', '1_2204', '1_2206', '1_2207', '1_2212', '1_2214', '1_2220', '1_2300', '1_2402', '1_2406', '1_2407', '1_2408', '1_2409', '1_2410', '1_2413', '1_2444', '1_2490', '1_2495', '1_2496', '1_2497', '1_2500', '1_2501', '1_2601', '1_2603', '1_3101', '1_3105', '1_3106', '1_3110', '1_3112', '1_3113', '1_3115', '1_3116', '1_3118', '1_3119', '1_3120', '1_3121', '1_3130', '1_3131', '1_3190', '1_3200', '1_3201', '1_3202', '1_3221', '1_3222', '1_3230', '1_3240', '1_3270', '1_3275', '1_3280', '1_3401', '1_3402', '1_3404', '1_3406', '1_3408', '1_3410', '1_3411', '1_3412', '1_3413', '1_3414', '1_3415', '1_3416', '1_3417', '1_3421', '1_3422', '1_3424', '1_3425', '1_3435', '1_3441', '1_3477', '1_3701', '1_3812', '1_3821', '1_3851', '1_3855', '1_3860', '1_3870', '1_3871', '1_3880', '1_4005', '1_4008', '1_4009', '1_4011', '1_4012', '1_4013', '1_4015', '1_4017', '1_4019', '1_4020', '1_4021', '1_4022', '1_4023', '1_4024', '1_4025', '1_4026', '1_4029', '1_4032', '1_4033', '1_4034', '1_4035', '1_4036', '1_4037', '1_4041', '1_4081', '1_4085', '1_4086', '1_4090', '1_4091', '1_4092', '1_4093', '1_4094', '1_4095', '1_4096', '1_4097', '1_4098', '1_4099', '1_4106', '1_4201', '1_4202', '1_4203', '1_4204', '1_4205', '1_4206', '1_4207', '1_4208', '1_42

<function __main__.plot_rt>

In [256]:
def net_stats(network_dir):
    stoptimes_df      = pd.read_csv(os.path.join(network_dir, 'stop_times.txt'), usecols=['trip_id','stop_id','stop_sequence'])
    stops_df          = pd.read_csv(os.path.join(network_dir, 'stops.txt'), usecols=['stop_id','stop_name','stop_lat','stop_lon'])
    trips_df          = pd.read_csv(os.path.join(network_dir, 'trips.txt'))

    num_stops  = stops_df.shape[0]
    num_st_stoptimes = len(list(set(stoptimes_df["stop_id"])))
    num_trips  = trips_df.shape[0]
    num_routes = len(list(set(trips_df["route_id"])))
    num_times_stop = stoptimes_df.shape[0]
    
    st = "Stoptime Stops, "+str(num_st_stoptimes)+"Stops, "+str(num_stops)+"\nTrips, "+str(num_trips)+"\nRoutes, "+str(num_routes)+"\nStopping, "+str(num_times_stop)+"\n"
    return st

In [257]:
HOME_DIR = r"/Users/elizabeth/Documents/urbanlabs/MTC/SHRP2/task4" 

@widgets.interact(network_dir = os.listdir(HOME_DIR))
def plot_net(network_dir):
    p = net_stats(os.path.join(HOME_DIR,network_dir))
    print p


interactive(children=(Dropdown(description=u'network_dir', options=('CHTS Run 7', 'network_draft1.14_fare', '.DS_Store', 'CHTS_fasttrips_demand_v0.5', 'readme-calibration.md', 'calibration-template.md', 'CHTS Run 3', 'SFO_SOMA_BART', 'trace_2201852', 'network_draft1.14_fare_clockhour', 'CHTS_fasttrips_demand_v0.5_sample', 'Route Choice with Stochastic Vehicle Arrivals-ELO.docx'), value='CHTS Run 7'), Output()), _dom_classes=('widget-interact',))

# Time Space

# Stops